Fill this cell as you locate the file

In [1]:
model_name = 'majo_no_tabitabi'
config_path = r'datasets\majo_no_tabitabi\config.json'
audio_dir = r'datasets\majo_no_tabitabi\audio'
filelist_train_path = r'datasets\majo_no_tabitabi\filelist_train.txt'
filelist_val_path = r'datasets\majo_no_tabitabi\filelist_val.txt'
speakers = ['elaina', 'fran', 'saya', 'sheila']
cleaners = 'korean_cleaners'

### Resample

In [2]:
import soundfile as sf
from tqdm import tqdm
import librosa
import os

sr_dst = 22050
for file in tqdm(os.listdir(audio_dir)):
    if file.endswith('.wav') or file.endswith('.mp3'):
        audio_path = os.path.join(audio_dir, file)
        audio, sr = librosa.load(audio_path, sr=None)
        audio_path = os.path.join(audio_dir, file.replace('.mp3', '.wav'))
        audio = librosa.resample(audio, sr, sr_dst)
        audio = librosa.to_mono(audio)
        sf.write(audio_path, audio, sr_dst)


### Preprocess

In [ ]:
# Single speaker
! python preprocess.py --text_index 1 --filelists {filelist_train_path} {filelist_val_path} --text_cleaners {cleaners}

In [3]:
# Mutiple speakers
! python preprocess.py --text_index 2 --filelists {filelist_train_path} {filelist_val_path} --text_cleaners {cleaners}

In [4]:
import json

filelist_train_clean_path = filelist_train_path.replace('.txt', '_cleaned.txt')
filelist_val_clean_path = filelist_val_path.replace('.txt', '_cleaned.txt')

with open(r"example\configs\korean.json", "r") as f:
    data = json.load(f)
data['data']['training_files'] = filelist_train_clean_path
data['data']['validation_files'] = filelist_val_clean_path
data['data']['text_cleaners'] = cleaners
data['data']['n_speakers'] = len(speakers)
data['speakers'] = speakers
with open(config_path, "w") as f:
    json.dump(data, f, indent=4)

### Train
If you want to use pretrained model, locate pretrained model at "./checkpoints/{model_name}/" with name "D_0.pth" and "G_0.pth" before you run following command.

In [ ]:
# Single speaker
print(f'python train.py -c {config_path} -m {model_name}')

In [ ]:
# Mutiple speakers
print(f'python train_ms.py -c {config_path} -m {model_name}')

### Tensorboard

In [ ]:
print(f'tensorboard --logdir=checkpoints/{model_name} --port=6006')